_______
Import everything we need:

In [85]:
import nbimporter
from wrangling import prep_dataframe
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import pydicom
import os
import numpy as np
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

___________
Get training data and convert it to pytorch tensors:

In [19]:
train_data = prep_dataframe("stage_2_train_mini", "stage_2_train.csv")

Error with file ID_1cde30e04.dcm


In [102]:
hemmorhage_train_data = train_data.loc[train_data['Hemmorhage']==True]
train_y = train_data['Hemmorhage']
train_x = train_data['Image']

#convert training images to numpy array
train_x = train_x.to_numpy()
train_x = np.array([np.array(x) for x in train_x])

#convert target data to numpy array
train_y = train_y.to_numpy()

#convert training images to pytorch tensors
train_x = train_x.reshape(315, 1, 512, 512)
train_x = torch.from_numpy(train_x)

#convert target data to pytorch tensor
train_y = train_y.astype('bool')
train_y = torch.from_numpy(train_y)

________
Define the CNN:

In [115]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.cnn_layers = Sequential(
            #convolution layer 1
            Conv2d(1, 4, kernel_size = 3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size = 2, stride=2),
            
            #convolution layer 2
            Conv2d(4, 4, kernel_size = 3, stride=1, padding=1),
            BatchNorm2d(4),
            ReLU(inplace=True),
            MaxPool2d(kernel_size = 2, stride=2),
        )
        
        self.linear_layers = Sequential(
            Linear(4 * 7 * 7, 10)
        )
        
    def forward(self, x):
        x = self.cnn_layers(x)
        return x

Define the loss function and optimizer for the model.
It seems like cross entropy loss is the standard loss function for CNNs.

In [119]:
cnn = CNN()
cnn_optimizer = torch.optim.Adam(cnn.parameters())
criterion = CrossEntropyLoss()

# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(cnn)

CNN(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=196, out_features=10, bias=True)
  )
)


Train the model.

In [120]:
def train(epoch, x_train, y_train):
    cnn.train()
    tr_loss = 0
    
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        x_val = x_val.cuda()
        y_val = y_val.cuda()
        
    #clear the gradients of the model parameters
    cnn_optimizer.zero_grad()
    
    #make a prediction
    output_train = cnn(x_train.int())
    
    #computing training loss
    loss_train = criterion(output_train, y_train)
    train_losses.append(loss_train)
    
    #update the weights of model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch % 5 == 0:
        print('Epoch: '+(epoch+1)+'\t'+'loss: '+loss_val)
    

In [121]:
n_epochs = 30
train_losses = []
for epoch in range(n_epochs):
    train(epoch, train_x, train_y)

RuntimeError: Expected object of scalar type Int but got scalar type Float for argument #3 'mat1' in call to _th_addmm_